In [4]:
import requests
from bs4 import BeautifulSoup

for num in range(100):
    URL = f"https://tsn.ua/news/page-{num}"

    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")

    news_links = []
    for link in soup.find_all("a", href=True):
        href = link["href"]
        if "/2025/" in href or "/2024/" or "/2023/" or "/2022/" in href: #фільтруємо новини за роком
            full_url = href if href.startswith("http") else f"https://tsn.ua/{href}"
            news_links.append(full_url)

print(f"len: {len(news_links)}\nNews_links: {news_links[:100]}")

len: 222
News_links: ['https://tsn.ua', 'https://tsn.ua/news', 'https://tsn.ua/youtube', 'https://tsn.ua/exclusive', 'https://tsn.ua/ukrayina', 'https://tsn.ua/politika', 'https://tsn.ua/ato', 'https://tsn.ua/glamur', 'https://tsn.ua/prosport', 'https://tsn.ua/lady', 'https://tsn.ua/zdorovya', 'https://tsn.ua/astrologiya', 'https://tsn.ua/video/video-novini', 'https://tsn.ua/food', 'https://tsn.ua/groshi', 'https://kyiv.tsn.ua', 'https://lviv.tsn.ua', 'https://tsn.ua/tsikavinki', 'https://tsn.ua/popular-tags', 'https://tsn.ua/vypusky', 'https://tsn.ua/svit', 'https://tsn.ua/tourism', 'https://tsn.ua/auto', 'https://tsn.ua/nauka_it', 'https://tsn.ua/dopomoga', 'https://tsn.ua/other', 'https://tsn.ua/authors', 'https://tsn.ua/online', 'https://tsn.ua/news', 'https://tsn.ua/ru/news', 'https://tsn.ua/en', 'https://tsn.ua/online', 'https://tsn.ua/news', 'https://tsn.ua/youtube', 'https://tsn.ua/exclusive', 'https://tsn.ua/ukrayina', 'https://tsn.ua/politika', 'https://tsn.ua/ato', 'https://

In [ ]:
import pandas as pd
import re

# Список для збору даних
news_data = []

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

unwanted_phrases = [
    "Читайте також:",
    "Підписуйтесь на наші канали у Telegram та Viber.",
    "Я дозволяю TSN.UA використовувати файли cookie",
    "© 2025 ТСН", "ТСН у соціальних мережах", "ТСН в социальных сетях", "© 2025 TSN"
]

for url in news_links:
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        title = soup.find("h1").text.strip() if soup.find("h1") else "No title found"
        time_tag = soup.find("time")
        publication_date = time_tag["datetime"] if time_tag else "No date found"

        paragraphs = soup.find_all("p")
        content_list = []

        for p in paragraphs:
            text = p.text.strip()

            if any(phrase in text for phrase in unwanted_phrases):
                continue

            if p.find("a"):
                continue

            if "Читайте також:" in text:
                break

            text = re.split(r"Нагадаємо|Читайте також", text)[0].strip()
            content_list.append(text)

        content = "\n".join(content_list)

        news_data.append({"url": url, "title": title, "date": publication_date, "content": content})
        # print(f"✅ Додано: {title[:50]}...")

    else:
        print(f"❌ Помилка для {url}, статус код: {response.status_code}")

# Створення DataFrame з отриманих даних
news_df = pd.DataFrame(news_data)

# Збереження у CSV
news_df.to_csv("news_dataset-2.csv", index=False, encoding="utf-8")
print("📂 Дані збережено в news_dataset.csv")


❌ Помилка для https://www.facebook.com/tsn.ua, статус код: 400
❌ Помилка для https://play.google.com/store/apps/details?id=ua.tsn.TCH_3&hl=ua, статус код: 404


C:\Users\Yuliia\AppData\Local\Temp\ipykernel_6960\3714257982.py:24: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(response.text, "html.parser")


❌ Помилка для https://tsn.ua/tel:+380444900101, статус код: 404
❌ Помилка для https://tsn.ua/mailto:feedback@tsn.ua, статус код: 404
📂 Дані збережено в news_dataset.csv


In [15]:
data = pd.read_csv("news_dataset.csv")
data.head(50)
data.dropna()

,url,title,date,content
27,https://tsn.ua/online,Новини ТСН 00:01,2020-06-22,Новини ТСН про події в Україні станом на 7 лют...
31,https://tsn.ua/tags/%D0%B2%D1%96%D0%B4%D0%BA%D...,відключення світла,2025-02-17T09:11:00+02:00,Читайте усі актуальні новини про відключення с...
32,https://tsn.ua/tags/%D0%B4%D0%BE%D0%BD%D0%B0%D...,дональд трамп,2025-02-17T11:51:00+02:00,Дональд Джон Трамп - 45-й та 47-й президент СШ...
34,https://tsn.ua/keywords/ozbroyennya-ta-armiya,Озброєння та армія,2025-01-29T16:33:00+02:00,Реалізується в партнерстві\nЧитайте статті про...
36,https://tsn.ua/keywords/poshireni-zapity,Актуальні запити,2024-11-05T22:24:00+02:00,Читайте статті та відповіді на поширені та акт...
...,...,...,...,...
230,https://tsn.ua/privacy-policy,Політика конфіденційності та захисту персональ...,No date found,Загальні положення\nКомпанія з великою повагою...
231,https://tsn.ua/cookie-policy,Використання файлів cookie,No date found,"Cookie — це текстовий файл або файли, що містя..."
232,https://tsn.ua/payment-privacy-policy,Політика конфіденційності. Фінансова допомога ...,No date found,"Всі терміни, що використовуються у цій Політиц..."
233,https://tsn.ua/payment-rules,Договір-оферта. Публічна оферта про надання бе...,No date found,Акцепт — повне і безумовне прийняття публічної...
